In [3]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from models.autoencoder import AE
import torch
from datasets.video_dataset import VideoDataset
import os
import pickle
from tqdm import tqdm

In [4]:
device = torch.device('cpu')
net = AE()
net.load_state_dict(torch.load('src/ae_epoch_19_loss_0.04.pt'))
net.eval()

AE(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
    (1): Hardswish()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): Hardswish()
    (5): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(4, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): Hardswish()
    (2): ConvTranspose2d(16, 8, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (3): Hardswish()
    (4): ConvTranspose2d(8, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (5): Tanh()
  )
)

In [5]:
def save_new_pickle(data, save_dir):
    c = 0
    while os.path.exists(os.path.join(save_dir, '%03d.pkl'%(c))):
        c += 1
    path = os.path.join(save_dir, '%03d.pkl'%(c))
    with open(path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
dataset = VideoDataset(r'E:\Gait\dataset', txt_list_file='src/test.txt', img_size=112, vid_len=40)
print(dataset.people)

['245', '107', '274', '296', '034', '322', '377', '053', '441', '318', '056', '310', '366', '221', '450', '132', '008', '005', '467', '168', '123', '023', '089', '376', '093', '044', '022', '287', '341', '423', '074', '065', '343', '203', '110', '048', '113', '120', '220', '149', '158', '169', '148', '270', '247', '440', '057', '073', '378', '397', '473', '131', '337', '075', '444', '095', '334', '208', '308', '174', '017', '436', '354', '329', '049', '468', '358', '263', '264', '104', '070', '071', '086']


In [7]:
save_dataset = 'gait_fc_dataset/test_p2'
os.makedirs(save_dataset, exist_ok=True)

with torch.no_grad():
    for i in tqdm(range(len(dataset))):
        vid, target = dataset.__getitem__(i)
        gei = np.zeros((112, 112))
        features = []
        for frame in  vid:
            code = net.encoder(frame.unsqueeze(0))
            code = code.view(-1).numpy().tolist()
            features.append(code)
            gei += frame[0].numpy()
        features = np.array(features)
        # print(features.shape)
        path_save = os.path.join(save_dataset, str(target))
        # print(path_save)
        os.makedirs(path_save, exist_ok=True)
        save_new_pickle({'ae_feat': features, 'gei': gei}, path_save)

        # print(gei)
        # plt.imshow(gei)
        # plt.show()
        # break

100%|██████████| 3947/3947 [04:55<00:00, 13.36it/s]
